To run this notebook you need:
- numpy
- scipy
- scikit-learn
- matplotlib
- pandas
- cython
- snakeviz
- a C compiler supporting OpenMP

In [ ]:
import time
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd
from sklearn.metrics import pairwise_distances_argmin
import matplotlib.pyplot as plt


%matplotlib inline
%load_ext cython
%load_ext snakeviz

# Cython tutorial
## An application with pairwise distances

### Profiling

Computing pairwise distances between points and centers is the critical part of K-means algorithm. Let's check that with the snakeviz profiling tool.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=100, n_init=1, algorithm='full', init='random')
X = np.random.random_sample((10000, 100))

%snakeviz kmeans.fit(X)

<br></br>
### The problem

Let `X` and `Y` be two sets of points. For all points in `X`, find its closest point in `Y`. Let's add a bit of parietal by doing it w.r.t the L1-Norm.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\forall x\in X, c(x) = \underset{y\in Y}{\operatorname{argmin}}{||x-y||_1}$

In [ ]:
def plot():
    Xv = np.array([[0.005*(i//200), 0.005*(i%200)] for i in range(40000)])
    Yv = np.random.RandomState(42).random_sample((20, 2))

    indices = pairwise_distances_argmin(Xv,Yv, metric='manhattan')
    plt.scatter(Xv[:,0], Xv[:,1], c=indices, s=10, marker='s')
    plt.scatter(Yv[:,0], Yv[:,1], color='red')
    plt.axis('equal')
    plt.rcParams['figure.figsize'] = [10, 10]

plot()

Helpers for accuracy testing and benchmarking

In [ ]:
def test(func, X, Y):
    true_indices = pairwise_distances_argmin(X, Y, metric='manhattan')
    indices = func(X, Y)
    if np.all(true_indices == indices):
        print('correct result \o/')
    else:
        print('incorrect result TT')

In [ ]:
benchs = pd.DataFrame(columns=['version', 'time(s)', 'speedup'])

def bench(func, X, Y, title):
    if title not in benchs:
        t = 0
        for _ in range(10):
            t_ = time.time()
            func(X, Y)
            t_ = time.time() - t_
            t += t_
        t /= 10
        
        i = benchs.shape[0]
        if i == 0:
            speedup = 1
        else:
            speedup = benchs[benchs['version']=='naive python']['time(s)'].values[0] / t
        benchs.loc[i] = [title, t, speedup]
        
        print(benchs)

<br></br>
Let's generate some data to benchmark our experiments.

In [ ]:
X = np.random.random_sample((10000, 100))
Y = np.random.random_sample((100, 100))

<br></br>
### Baseline: naive python

In [ ]:
def naive_python(X, Y):
    distances = np.empty((X.shape[0], Y.shape[0]))
    for i, x in enumerate(X):
        for j, y in enumerate(Y):
            distances[i, j] = np.abs(x - y).sum()
    
    return np.argmin(distances, axis=1)

In [ ]:
test(naive_python, X, Y)

In [ ]:
bench(naive_python, X, Y, 'naive python')

In [ ]:
%memit naive_python(X, Y)

<br></br>
### Goal

Scipy `cdist` is a function implemented in C which computes the pairwise distances between two sets of points.

In [ ]:
def scipy_pairwise(X, Y):
    distances = cdist(X, Y, metric='cityblock')
    return np.argmin(distances, axis=1)

In [ ]:
bench(scipy_pairwise, X, Y, 'cdist')

In [ ]:
%memit scipy_pairwise(X, Y)

There's a helper in scikit-learn which does the distances and argmin computations (using scipy's cdist under the hood).

In [ ]:
def sklearn_pairwise(X, Y):
    return pairwise_distances_argmin(X, Y, metric='manhattan')

In [ ]:
bench(sklearn_pairwise, X, Y, 'sklearn')

<br></br>
### numpy only ?

Using numpy only requires making use of broadcasting, which may increase memory consumption.

In [ ]:
def naive_numpy(X, Y):
    distances = np.abs(X[:,:,np.newaxis] - Y.T[np.newaxis,:,:]).sum(axis=1)
    
    return np.argmin(distances, axis=1)

In [ ]:
test(naive_numpy, X, Y)

In [ ]:
bench(naive_numpy, X, Y, 'naive numpy')

In [ ]:
%memit naive_numpy(X, Y)

<br></br>
### Cython

In [ ]:
%%cython -f
import numpy as np


def cython_pairwise(X, Y):
    distances = np.empty((X.shape[0], Y.shape[0]))
    for i, x in enumerate(X):
        for j, y in enumerate(Y):
            distances[i, j] = np.abs(x - y).sum()
    
    return np.argmin(distances, axis=1)

In [ ]:
test(cython_pairwise, X, Y)

In [ ]:
bench(cython_pairwise, X, Y, 'cython nothing')

<br></br>
## More cython

### Fused types

In [ ]:
%%cython -f
import numpy as np
from libc.math cimport sqrt


cpdef double euclidean_distance(double[::1] x, double[::1] y):
    cdef:
        int i
        double res = 0

    for i in range(x.shape[0]):
        res += (x[i] - y[i]) * (x[i] - y[i])

    return sqrt(res)

In [ ]:
x = np.ones(100)
y = np.zeros(100)
euclidean_distance(x, y)

<br></br>
### Paralellism with OpenMP

First a simple parallel loop

In [ ]:
%%cython

In [ ]:
test(cython_pairwise, X, Y)

In [ ]:
bench(cython_pairwise, X, Y, 'cython prange')

<br></br>
Now a parallel loop with local buffer

In [ ]:
%%cython -c=-fopenmp -f
#cython: wraparound=False, boundscheck=False
import numpy as np
from cython.parallel cimport parallel, prange
from libc.math cimport fabs
from libc.stdlib cimport malloc, free


cdef double manhattan_distance(double *x, double *y, int n) nogil:
    cdef:
        int i
        double res = 0
        int k = n // 4
        int r = n % 4
    
    for i in range(k):
        res += (fabs(x[0] - y[0])
               +fabs(x[1] - y[1])
               +fabs(x[2] - y[2])
               +fabs(x[3] - y[3]))
        x += 4; y += 4
    
    for i in range(r):
        res += fabs(x[i] - y[i])
    
    return res


cdef void manhattan_distances_chunked(double *X, int nx,
                                      double *Y, int ny,
                                      int n_features,
                                      double *distances) nogil:
    cdef:
        int i, j
        
    for i in range(nx):
        for j in range(ny):
            distances[i * ny + j] = manhattan_distance(X + i * n_features, Y + j * n_features, n_features)
                

cdef void argmin_chunked(double *distances, int m, int n, int *indices) nogil:
    cdef:
        int i, j
        int best_j
        double min_dist
        
    for i in range(m):
        min_dist = distances[i * n] 
        best_j = 0
        
        for j in range(1, n):
            if distances[i * n + j] < min_dist:
                min_dist = distances[i * n + j]
                best_j = j
        
        indices[i] = best_j
            


def cython_parallel_pairwise(double[:, ::1] X, double[:, ::1] Y):
    cdef:
        int i
        int n_samples_X = X.shape[0]
        int n_samples_Y = Y.shape[0]
        int n_features = X.shape[1]
        
        int[::1] indices = np.empty(n_samples_X, dtype=np.int32)
        
        double *local_buffer
        int nx = 100
        int n_chunks = n_samples_X // nx

    with nogil, parallel():
        local_buffer = <double *> malloc(nx * n_samples_Y * sizeof(double))
        
        for i in prange(n_chunks):
            manhattan_distances_chunked(&X[i * nx, 0], nx,
                                        &Y[0, 0], n_samples_Y,
                                        n_features,
                                        local_buffer)
        
            argmin_chunked(local_buffer, nx, n_samples_Y, &indices[i * nx])
        
        free(local_buffer)
            
    return np.asarray(indices)

In [ ]:
test(cython_parallel_pairwise, X, Y)

In [ ]:
bench(cython_parallel_pairwise, X, Y, 'cython prange buffer')